In [1]:
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
from koopmanlib.dictionary import PsiNN
from koopmanlib.solver import KoopmanDLSolver

2025-07-01 02:39:15.967431: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-01 02:39:16.095944: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-01 02:39:16.157706: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-01 02:39:16.172860: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-01 02:39:16.255074: I tensorflow/core/platform/cpu_feature_guar

In [2]:
dim = 2
hidden_dim = 50
input_dim = 0
n_feature = 48
num_epochs = 100
batch_size = 2048
n_train = 50000
n_valid = 5000
n_test = 5000
learning_rate = 1e-3

In [3]:
X_train = pd.read_csv('Van_train.csv', header=None).values
X_valid = pd.read_csv('Van_valid.csv', header=None).values
X_test = pd.read_csv('Van_test.csv', header=None).values

length = X_train.shape[1] // n_train
H_train = []
for i in range(n_train):
    H_train.append(X_train[:, i*length:(i+1)*length])
H_train = np.stack([H_train[idx].T for idx in range(n_train)], axis=0)
H_valid = []
for i in range(n_valid):
    H_valid.append(X_valid[:, i*length:(i+1)*length])
H_valid = np.stack([H_valid[idx].T for idx in range(n_valid)], axis=0)
H_test = []
for i in range(n_test):
    H_test.append(X_test[:, i*length:(i+1)*length])
H_test = np.stack([H_test[idx].T for idx in range(n_test)], axis=0)

In [ ]:
error_temp = []

data_train = [H_train[:, 0:-1,:].reshape(-1, dim+input_dim), H_train[:, 1:,:].reshape(-1, dim+input_dim)]
data_valid = [H_valid[:, 0:-1,:].reshape(-1, dim+input_dim), H_valid[:, 1:,:].reshape(-1, dim+input_dim)]
basis_function = PsiNN(layer_sizes=[hidden_dim, hidden_dim, hidden_dim], n_psi_train=dim+n_feature)
solver = KoopmanDLSolver(dic=basis_function,
                        target_dim=dim+input_dim,
                        reg=0.0)
solver.build(data_train=data_train,
                 data_valid=data_valid,
                 epochs=num_epochs,
                 batch_size=batch_size,
                 lr=learning_rate,
                 log_interval=1,
                 lr_decay_factor=0.92)
for j in range(n_test):
    x_traj = H_test[j, :,:]
    x0_test = x_traj[0]
    x0_test = x0_test.reshape(-1, x0_test.shape[-1])
    x_est_traj_DL = solver.predict(x0_test, length-1)
    error_temp.append((x_est_traj_DL[:, :dim]-x_traj[1:, :dim]) ** 2)
error = np.mean(np.array(error_temp))
    
df = pd.DataFrame([error])
df.to_csv("Van_EDMDDL.csv", index=False)